In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import os

input_path = r'C:\Users\Saman\Desktop\enamad.xlsx'
output_path = "seo_content_info.xlsx"
status_path = "progress_status.txt"

df = pd.read_excel(input_path)
#df = df.head(10)

# Index to figure out from witch row we should start
start_index = 0
if os.path.exists(status_path):
    with open(status_path, 'r') as f:
        saved_index = f.read()
        if saved_index.isdigit():
            start_index = int(saved_index)

#build dataframe for columns
if not os.path.exists(output_path):
    output_df = pd.DataFrame(columns=[
        "Company", "URL", "Status", "Resp1_Time", "Resp2_Time", "Avg_Response_Time",
        "Title", "H1", "H2_Tags", "Meta_Description", "Social_Links", "Strong Tags", "Structured_Data"
    ])
    output_df.to_excel(output_path, index=False)

for index, row in df.iloc[start_index:].iterrows():
    try:
        company = row['Company']
        domain = str(row['Domain']).strip()
        url = "https://" + domain if not domain.startswith("http") else domain

        response_times = []
        status = None
        response = None

        for _ in range(2):
            try:
                start = time.time()
                response = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
                end = time.time()
                duration = round(end - start, 3)
                response_times.append(duration)
                status = response.status_code
            except:
                response_times.append(None)

        resp1 = response_times[0] if len(response_times) > 0 else None
        resp2 = response_times[1] if len(response_times) > 1 else None
        sta = status if status else None
        valid_times = [t for t in response_times if t is not None]
        avg_time = round(sum(valid_times) / len(valid_times), 3) if valid_times else None

        title_tag = h1 = meta_description = structured_data = "NOT FOUND"
        email = phone = social_links = h2_tags = "NOT FOUND"

        if response and response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            title_tag = soup.title.string.strip() if soup.title and soup.title.string else "NOT FOUND"
            h1_tag = soup.find("h1")
            h1 = h1_tag.text.strip() if h1_tag else "NOT FOUND"

            h2s = [h2.get_text(strip=True) for h2 in soup.find_all('h2') if h2.get_text(strip=True)]
            h2_tags = ' | '.join(h2s[:5]) if h2s else "NOT FOUND"

            desc_tag = soup.find("meta", attrs={"name": "description"})
            meta_description = desc_tag['content'] if desc_tag and 'content' in desc_tag.attrs else "NOT FOUND"

            socials = [a['href'] for a in soup.find_all('a', href=True)
                       if any(s in a['href'] for s in ['instagram.com', 'linkedin.com', 'facebook.com', 'twitter.com','telegram.com','aparat.ir'])]
            social_links = ', '.join(socials) if socials else "NOT FOUND"

            strong = [s.get_text(strip=True) for s in soup.find_all('strong') if s.get_text(strip=True)]
            strong_tags = '|'.join(strong[:5]) if strong else "NOT FOUND"

            schema_tag = soup.find("script", type="application/ld+json")
            structured_data = schema_tag.text.strip() if schema_tag else "NOT FOUND"

        # add row immedaitly 
        new_row = pd.DataFrame([{
            "Company": company,
            "URL": url,
            "Status": sta,
            "Resp1_Time": round(resp1, 3) if resp1 else "FAILED",
            "Resp2_Time": round(resp2, 3) if resp2 else "FAILED",
            "Avg_Response_Time": round(avg_time, 3) if avg_time else "FAILED",
            "Title": title_tag,
            "H1": h1,
            "H2_Tags": h2_tags,
            "Meta_Description": meta_description,
            "Social_Links": social_links,
            "Strong Tags" : strong_tags,
            "Structured_Data": structured_data,
        }])
        
        # append to excel without overwriting
        with pd.ExcelWriter(output_path, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            # we need to read from excel file
            existing_df = pd.read_excel(output_path)
            updated_df = pd.concat([existing_df, new_row], ignore_index=True)
            updated_df.to_excel(writer, index=False)

        # save Index for retry antoher time
        with open(status_path, 'w') as f:
            f.write(str(index + 1))

    except Exception as e:
        print(f"Error on row {index}: {e}")
    
        break

print("✅ Done. Results saved in seo_content_info.xlsx")


Error on row 106: [Errno 13] Permission denied: 'seo_content_info.xlsx'
✅ Done. Results saved in seo_content_info.xlsx
